In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import random
from PIL import Image
import numpy as np
import torch
from generators import Generator, GeneratorSN

# Load a trained model

In [ ]:
generator = GeneratorSN().cuda()
generator.load_state_dict(torch.load('models/run02_generator.pth'))

# Get a random image

In [ ]:
folder = '/mnt/datasets/celeba_good_faces/train/'
names = os.listdir(folder)
n = random.choice(names)
image = Image.open(os.path.join(folder, n))
image

# Generate another version

In [ ]:
x = torch.Tensor(np.array(image)/255.0).unsqueeze(0)
x = x.permute(0, 3, 1, 2).cuda()
restored_x = generator(x).detach()

In [ ]:
images1 = np.concatenate(np.transpose((255.0*x).cpu().numpy(), [0, 2, 3, 1]), axis=0)
images2 = np.concatenate(np.transpose((255.0*restored_x).cpu().numpy(), [0, 2, 3, 1]), axis=0)
result = Image.fromarray(np.concatenate([images1, images2], axis=1).astype('uint8'))
result

In [ ]:
result.convert('L')

# Plot losses

In [ ]:
import json
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
with open('losses_run00.json', 'r') as f:
    losses = json.load(f)
    
loss_names = list(losses[0].keys())

In [ ]:
plt.figure(figsize=(14, 7))

for n in loss_names:
    values = [d[n] for d in losses]
    plt.plot(values, label=n)

plt.xlabel('step')
plt.legend();